In [428]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tf.random.set_seed(1)



import pandas as pd
X_submit = pd.read_csv('https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/test.csv')
X = pd.read_csv('https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/train.csv')
X_test = pd.read_csv('https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/test.csv')
X.drop(columns=['id','Product ID'], inplace=True)
X_test.drop(columns=['id','Product ID'],inplace=True)

In [429]:
oversample = RandomOverSampler(random_state=1, sampling_strategy=.1)

In [430]:
y = X['Machine failure']
X.drop(columns=['Machine failure'], inplace = True)

In [431]:
y.value_counts()

0    134281
1      2148
Name: Machine failure, dtype: int64

In [432]:
X_submit.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0


In [433]:
# X, y = oversample.fit_resample(X, y)


In [434]:
y.value_counts()

0    134281
1      2148
Name: Machine failure, dtype: int64

In [435]:

# X = pd.get_dummies(X)
# prod_ID = X['Product ID'].tolist()
# machine_type = X['Type'].tolist()
# le = LabelEncoder()
# X['Product ID'] = le.fit_transform(prod_ID)
# X['Type'] = le.fit_transform(machine_type)
# prod_ID
# le.classes_
# prod_ID = le.fit(X['Product ID'])
# type = le.fit(X['Type'])
# X['Product ID'] = prod_ID
# X['Type'] = machine_type

In [436]:
# X.shape
X.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,L,300.6,309.6,1596,36.1,140,0,0,0,0,0
1,M,302.6,312.1,1759,29.1,200,0,0,0,0,0
2,L,299.3,308.5,1805,26.5,25,0,0,0,0,0
3,L,301.0,310.9,1524,44.3,197,0,0,0,0,0
4,M,298.0,309.0,1641,35.4,34,0,0,0,0,0


In [437]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [438]:
X.tail()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
136424,M,300.1,311.4,1530,37.5,210,0,0,0,0,0
136425,H,297.5,308.5,1447,49.1,2,0,0,0,0,0
136426,L,300.5,311.8,1524,38.5,214,0,0,0,0,0
136427,L,301.7,310.9,1447,46.3,42,0,0,0,0,0
136428,L,296.9,308.1,1557,39.3,229,0,0,0,0,0


In [439]:
# X.nunique()

In [440]:
# X_test.nunique()

In [441]:
X_combine = pd.concat([X, X_test])
# dummy_df = pd.get_dummies(X_combine)
# prod_ID = X_combine['Product ID'].tolist()
machine_type = X_combine['Type'].tolist()
le = LabelEncoder()
# X_combine['Product ID'] = le.fit_transform(prod_ID)
X_combine['Type'] = le.fit_transform(machine_type)
i = X.shape[0]
X_combine_train = X_combine.iloc[0:i]
X_combine_test = X_combine.iloc[i:]
X_combine_train, y = oversample.fit_resample(X_combine_train, y)
# X_train_dropped = X[['Product ID','Type']]
# X_test_dropped = X_test[['Product ID','Type']]
# X_train.drop(columns=['Product ID','Type'], inplace=True)
# X_test.drop(columns=['Product ID','Type'], inplace=True)
# X.dtypes

In [442]:
X_combine_train.shape


(147709, 11)

In [443]:
scaler = StandardScaler()

X_Scaler = scaler.fit(X_combine_train)

X_train_scaled = X_Scaler.transform(X_combine_train)
X_test_scaled = X_Scaler.transform(X_combine_test)

# X_Scaler = scaler.fit(X_train)

# X_train_scaled = X_Scaler.transform(X_train)
# X_test_scaled = X_Scaler.transform(X_test)

In [444]:
# type(X_train_scaled[0])
# X_train.shape

In [445]:
# X_train = np.concatenate((X_train_scaled, X_dropped), axis = 1)
# for i in range(0,X_train.shape[0]):
#     np.concatenate(X_train_scaled[i],X_train_dropped["Product ID"][i])
#     np.concatenate(X_train_scaled[i],X_train_dropped["Type"][i])

# for i in range(0, X_test.shape[0]):
#     np.concatenate(X_test_scaled[i],X_train_dropped['Product ID'][i])
#     np.concatenate(X_test_scaled[i],X_train_dropped['Type'][i])

In [446]:
# n = len(dummy_df.columns)

n= len(X.columns)
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units = 10, activation = 'relu', input_dim = n))

nn.add(tf.keras.layers.Dense(units = 3, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

nn.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 10)                120       
_________________________________________________________________
dense_43 (Dense)             (None, 3)                 33        
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 4         
Total params: 157
Trainable params: 157
Non-trainable params: 0
_________________________________________________________________


In [447]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [448]:
# X_temp = np.asarray(dummy_train).astype(np.float32)
history = nn.fit(X_combine_train, y, epochs = 100)

Epoch 1/100


4616/4616 [==============================] - 2s 389us/step - loss: 0.5877 - accuracy: 0.9137
Epoch 2/100
4616/4616 [==============================] - 2s 389us/step - loss: 0.1951 - accuracy: 0.9393
Epoch 3/100
4616/4616 [==============================] - 2s 373us/step - loss: 0.1255 - accuracy: 0.9657
Epoch 4/100
4616/4616 [==============================] - 2s 378us/step - loss: 0.1114 - accuracy: 0.9715
Epoch 5/100
4616/4616 [==============================] - 2s 367us/step - loss: 0.1062 - accuracy: 0.9744
Epoch 6/100
4616/4616 [==============================] - 2s 378us/step - loss: 0.1047 - accuracy: 0.9757
Epoch 7/100
4616/4616 [==============================] - 2s 374us/step - loss: 0.1062 - accuracy: 0.9758
Epoch 8/100
4616/4616 [==============================] - 2s 375us/step - loss: 0.1031 - accuracy: 0.9766
Epoch 9/100
4616/4616 [==============================] - 2s 378us/step - loss: 0.1011 - accuracy: 0.9768
Epoch 10/100
4616/4616 [==============================] - 2s 382us/

In [458]:
# predict = nn.predict_classes(X_test)
# print(confusion_matrix(y_test, predict))


In [450]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     33571
           1       0.98      0.79      0.87      3357

    accuracy                           0.98     36928
   macro avg       0.98      0.89      0.93     36928
weighted avg       0.98      0.98      0.98     36928



In [451]:

# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [461]:
predict = nn.predict_classes(X_test_scaled)
predict.reshape(-1,1)
# predict[2][0]
predict_list = []
for i in predict:
    predict_list.append(predict[i][0][0])
id_numbers = X_submit['id'].tolist()
submission = pd.DataFrame({'id':id_numbers, 'Machine failure':predict_list})

In [462]:
# submission[submission['Machine failure'] == 0]

submission.to_csv('./nn_prediction_submission.csv', index = False)

In [ ]:
# predictions = []
# for i in predict:
#     predictions.append(predict[i][0][0])
# predictions_df = pd.DataFrame({'prediction':predictions})


In [ ]:
# predictions_df.value_counts()